In [4]:
from pumautil.notification import ServiceInput
from wafs import wafs
import xarray as xr
import numpy as np
import sparse
import matplotlib.pyplot as plt

xr.set_options(display_expand_data=False)
%matplotlib inline

In [ ]:
metadata_config["runtime"]["infile_Wind_temp"] = '../tests/test_data/ingress_met_model_WAF_2022_06_15_KWBC_00_Wind_temp_20220615_0000f06.grib2'
metadata_config["runtime"]["infile_ice"] = '../tests/test_data/ingress_met_model_WAF_2022_06_15_KWBC_00_ice_20220615_0000f06_ice.grib2'
metadata_config["runtime"]["infile_turb"] = '../tests/test_data/ingress_met_model_WAF_2022_06_15_KWBC_00_turb_20220615_0000f06_turb.grib2'

In [ ]:
import importlib

importlib.reload(wafs)

In [1]:
# wafs.run(ServiceInput)

In [ ]:
full_run_through_test_file = "wafs/20220615T000000Z.nc"
ds_final = xr.open_dataset(full_run_through_test_file)
ds_final

In [ ]:
ds_test_grib = xr.open_dataset(
    wafs.metadata_config["runtime"]["infile_wind"],
    engine="cfgrib",
    backend_kwargs={"filter_by_keys": {"typeOfLevel": "isobaricInhPa"}},
)

In [ ]:
ds_test_grib_regridded = wafs.regrid_ds(ds_test_grib)
da_test_grib_temp_celcius = ds_test_grib_regridded["t"] - 273.15
da_final_temp_celcius = ds_final["air_temperature"]

In [ ]:
# 700 hpa = 10_000 feet
(ds_test_grib.t.sel(isobaricInhPa=700) - 273.15).plot(vmin=-20, vmax=20)

In [ ]:
da_test_grib_temp_celcius.sel(isobaricInhPa=700).plot(vmin=-20, vmax=20)

In [ ]:
da_final_temp_celcius.isel(flight_altitude_wind_temp=2).plot(vmin=-20, vmax=20)

In [ ]:
da_test_grib_temp_celcius

In [ ]:
da_final_temp_celcius

In [ ]:
# FL300 on https://www.aviationweather.gov/wafs
# flight level 300 means 30,000 foot
# corresponds to flight_altitude_ft=27
# also in knots which are 1.94384 * the value in m/s
# same colour bar scale as the website
(
    ds_final["wind_speed_mps"].interp(
        flight_altitude_wind_temp=30000, kwargs={"fill_value": "extrapolate"}
    )
    * 1.94384
).plot(vmin=60, vmax=160)

In [ ]:
# FL300 on https://www.aviationweather.gov/wafs
# flight level 300 means 30,000 foot
ds_final["wind_direction"].interp(
    flight_altitude_wind_temp=30000, kwargs={"fill_value": "extrapolate"}
).plot(cmap="twilight")

In [ ]:
# FL390 on https://www.aviationweather.gov/wafs
# flight level 390 means 39,000 foot
# corresponds to flight_altitude_turb=-2
ds_final["eddy_dissipation_parameter"].isel(flight_altitude_turb=-2).plot()

In [ ]:
# FL100 on https://www.aviationweather.gov/wafs
# flight level 100 means 10,000 foot
# corresponds to flight_altitude_ice=2
ds_final["icing_severity_index"].isel(flight_altitude_ice=1).plot(vmin=-1, vmax=4)

In [ ]:
ds_final["air_temperature"].interp(
    flight_altitude_ft=49000, kwargs={"fill_value": "extrapolate"}
).plot()

In [ ]:
from pumautil.notification import ServiceInput
run(ServiceInput)

Old `test_run` function (may be useful for QA/QC)

```python
@pytest.mark.slow
def test_run(
    self,
    ds_test_grib: xr.Dataset,
    print_results: bool = True,
    assert_criteria_to_thresholds: bool = False,
) -> None:
    """Assert that interpolated values compare well to raw data.

    Currently, only for air temperature.

    The altitudes are matched between hPa and feet.
    Approximate pairs are found between the original and final datasets.
    The difference and ratio of values for each pair is calculated.
    The differences and ratios are summed for when they are within a range criteria.

    Parameters
    ----------
    ds_test_grib : xr.Dataset
        the test raw data
    print_results : bool, default True
        if wish to print the final results for ratios and differences
    assert_criteria_to_thresholds: bool, default False
        if wish to assert criteria against specific thresholds, failing the test if not met
    """
    wafs.run(
        ServiceInput
    )  # TODO: change this to notification, which is receive_notification()?
    filename = "20220615T000000Z"
    if os.path.exists(f"{filename}.nc"):
        using_format = "netcdf"
        filename_with_format = f"{filename}.nc"
        ds_final = xr.open_dataset(filename_with_format)
    else:
        using_format = "zarr"
        filename_with_format = f"{filename}.zarr"
        ds_final = xr.open_zarr(filename_with_format)

    ds_test_grib_regridded = wafs.regrid_ds(ds_test_grib)

    da_test_grib_temp_celcius = ds_test_grib_regridded["t"] - 273.15
    da_final_temp_celcius = ds_final["air_temperature"]

    approximate_altitude_hpa_vs_feet = {}
    for pressure_hpa in ds_test_grib.isobaricInhPa.values:
        height_pressure_feet_rounded = round(
            mpcalc.pressure_to_height_std(pressure_hpa * units("hPa"))
            .to(units("ft"))
            .magnitude,
            -3,
        )
        approximate_altitude_hpa_vs_feet[str(int(pressure_hpa))] = str(
            int(height_pressure_feet_rounded)
        )

    indices_hpa = {}
    for pressure_hpa in ds_test_grib.isobaricInhPa.values:
        index_hpa = np.argwhere(
            ds_test_grib.isobaricInhPa.values == pressure_hpa
        )
        indices_hpa[str(int(pressure_hpa))] = str(int(index_hpa))

    indices_feet = {}
    for (
        altitude_feet
    ) in da_final_temp_celcius.flight_altitude_wind_temp.values:
        index_feet = np.argwhere(
            da_final_temp_celcius.flight_altitude_wind_temp.values
            == altitude_feet
        )
        indices_feet[str(int(altitude_feet))] = str(int(index_feet))

    ratios = [round(ratio, 1) for ratio in np.linspace(0.1, 0.5, 5)]
    percentages_of_ratios_within = {str(ratio): [] for ratio in ratios}
    diffs = [int(diff) for diff in np.linspace(1, 5, 5)]
    percentages_of_diffs_within = {str(diff): [] for diff in diffs}

    for (
        altitude_hpa,
        altitude_feet,
    ) in approximate_altitude_hpa_vs_feet.items():
        if (
            float(altitude_feet) < 36089
        ):  # the barometric conversion formula only holds for this
            index_hpa = int(indices_hpa[altitude_hpa])
            index_feet = int(indices_feet[altitude_feet])

            diff_temp = da_test_grib_temp_celcius.isel(
                isobaricInhPa=index_hpa
            ) - da_final_temp_celcius.isel(
                flight_altitude_wind_temp=index_feet
            )
            ratio_temp = da_test_grib_temp_celcius.isel(
                isobaricInhPa=index_hpa
            ) / da_final_temp_celcius.isel(
                flight_altitude_wind_temp=index_feet
            )

            for ratio in ratios:
                ratio_temp_within = ratio_temp.where(
                    cond=np.abs(ratio_temp) > (1.0 - ratio)
                ).where(cond=np.abs(ratio_temp) < (1.0 + ratio))

                percentage_of_values_within = 1 - (
                    ratio_temp_within.isnull().sum().values
                    / (
                        ratio_temp_within.lon.shape[0]
                        * ratio_temp_within.lat.shape[0]
                    )
                )

                percentages_of_ratios_within[str(ratio)].append(
                    percentage_of_values_within
                )

            for diff in diffs:
                diff_temp_within = diff_temp.where(
                    cond=np.abs(diff_temp) < diff
                )

                percentage_of_diffs_within = 1 - (
                    diff_temp_within.isnull().sum().values
                    / (
                        diff_temp_within.lon.shape[0]
                        * diff_temp_within.lat.shape[0]
                    )
                )

                percentages_of_diffs_within[str(diff)].append(
                    percentage_of_diffs_within
                )

    if print_results:
        for ratio, values in percentages_of_ratios_within.items():
            print(
                f"Percentage of ratios within {100 * float(ratio):0.0f}%: {100 * np.mean(values):0.0f}%"
            )

        for diff, values in percentages_of_diffs_within.items():
            print(
                f"Percentage of diffs within {float(diff):0.1f} degrees C: {100 * np.mean(values):0.0f}%"
            )

    if assert_criteria_to_thresholds:
        assert np.mean(percentages_of_ratios_within["0.1"]) > 0.6
        assert np.mean(percentages_of_ratios_within["0.2"]) > 0.85
        assert np.mean(percentages_of_ratios_within["0.3"]) > 0.9
        assert np.mean(percentages_of_ratios_within["0.4"]) > 0.9
        assert np.mean(percentages_of_ratios_within["0.5"]) > 0.95

        assert np.mean(percentages_of_diffs_within["1"]) > 0.4
        assert np.mean(percentages_of_diffs_within["2"]) > 0.6
        assert np.mean(percentages_of_diffs_within["3"]) > 0.7
        assert np.mean(percentages_of_diffs_within["4"]) > 0.8
        assert np.mean(percentages_of_diffs_within["5"]) > 0.9

    if using_format == "netcdf":
        os.remove(filename_with_format)
    elif using_format == "zarr":
        shutil.rmtree(filename_with_format)
```